In [87]:
import pandas as pd
import numpy as np
import datetime
path = "task_1.csv"

In [96]:
#dateparse = lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S')
tmp = pd.read_csv(
    path,
    encoding = "cp1251",
    sep=";",
    on_bad_lines='skip',
    header=0,
    names=[
        "number",
        "type",
        "service",
        "priority",
        "status",
        "dt_create",
        "dt_solve",
        "dt_close",
        "firm",
        "group",
        "set",
        "client",
        "topic",
        "sla"
    ],

)


#####  1. Рассчитайте предварительные показатели для 1 квартала 2022 и подготовьте динамику для сравнения за предыдущие периоды

In [97]:

tmp[["dt_create", "dt_solve", "dt_close"]]  = tmp[["dt_create", "dt_solve", "dt_close"]].astype('datetime64')
df = tmp.loc[
        (tmp["type"] == "Инцидент") &
        (tmp["client"] != 'Мониторинг') &
        (tmp["sla"] == 'Да') &
        (tmp["service"].isin(["НОП", "TelcoCloud", "Платформа Цифровых Продуктов", "РТК-Экраны", "ЗСПД ЕГИСЗ", "ППЭ"])) &
        (tmp["firm"] == 'ДЭФИР') 
    ]
df

df['quoter'] = df['dt_solve'].dt.quarter
pd.pivot_table(df, values='number',  index='service', columns='quoter', aggfunc='count')


C:\Users\KurzanovAV\AppData\Local\Temp\ipykernel_23508\1593877431.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quoter'] = df['dt_solve'].dt.quarter


quoter,1,2,3,4
service,,,,
TelcoCloud,4,6,5,7
ЗСПД ЕГИСЗ,9,3,12,15
НОП,183,262,221,270
ППЭ,5,4,17,13
Платформа Цифровых Продуктов,6,11,3,7
РТК-Экраны,4,6,13,7


##### 2. Рассчитайте, какая группа специалистов ДЭФИР решила больше всего обращений за 2 квартал 2021 года с учетом, что решившая группа остается заполненной в поле Назначенная группа.

In [104]:
df.loc[df['quoter'] == 2, ['number', 'group']].groupby('group').count()

,number
group,
Группа 1,53
Группа 2,67
Группа 3,57
Группа 4,50
Группа 5,65


In [90]:
pd.pivot_table(df, values='number',  index='service', columns='period', aggfunc='count')


C:\Users\KurzanovAV\AppData\Local\Temp\ipykernel_23508\417649888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'period'] = np.where((df.loc[:, "dt_solve"] >= pd.to_datetime('2021-07-01')) & (df.loc[:, "dt_solve"] < pd.to_datetime('2021-10-01')), 3, 0)
C:\Users\KurzanovAV\AppData\Local\Temp\ipykernel_23508\417649888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'period'] = np.where((df.loc[:, "dt_solve"] >= pd.to_datetime('2021-10-01')) & (df.loc[:, "dt_solve"] < pd.to_datetim

period,1,3,4
service,,,
TelcoCloud,4,5,7
ЗСПД ЕГИСЗ,9,12,15
НОП,183,221,270
ППЭ,5,17,13
Платформа Цифровых Продуктов,6,3,7
РТК-Экраны,4,13,7


In [91]:
df['quoter'] = df['dt_solve'].dt.quarter
df[df['quoter'] > 1]


C:\Users\KurzanovAV\AppData\Local\Temp\ipykernel_23508\167290093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quoter'] = df['dt_solve'].dt.quarter


,number,type,service,priority,status,dt_create,dt_solve,dt_close,firm,group,set,client,topic,sla,period,quoter
850,INC00001333549,Инцидент,НОП,3 - Средний,Закрыт,2021-07-01 09:25:12,2021-07-01 09:46:28,2021-07-07 00:26:12,ДЭФИР,Группа 3,Илья А,Клиент 720,не отражается камера в НОП,Да,3,3
851,INC00001334086,Инцидент,НОП,3 - Средний,Закрыт,2021-07-01 10:45:03,2021-07-01 11:18:58,2021-07-06 22:48:45,ДЭФИР,Группа 3,Даниил А,Клиент 831,Загрузка ЦП Toliman,Да,3,3
853,INC00001326688,Инцидент,НОП,3 - Средний,Закрыт,2021-06-26 17:55:09,2021-07-01 12:22:16,2021-08-31 03:04:47,ДЭФИР,Группа 4,Михаил Г,Клиент 343,[#CASE-TT-309682] Деградация сервиса услуга не...,Да,3,3
854,INC00001326960,Инцидент,НОП,3 - Средний,Закрыт,2021-06-26 23:55:03,2021-07-01 12:22:54,2021-08-31 03:02:27,ДЭФИР,Группа 2,Михаил Г,Клиент 578,[#CASE-TT-309682] Деградация сервиса услуга не...,Да,3,3
855,INC00001327206,Инцидент,НОП,3 - Средний,Закрыт,2021-06-27 21:50:17,2021-07-01 12:23:21,2021-08-31 03:04:37,ДЭФИР,Группа 1,Михаил Г,Клиент 767,"[#CASE-TT-309837] Деградация сервиса, услуга н...",Да,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414,INC00001444519,Инцидент,ППЭ,3 - Средний,Закрыт,2021-12-30 03:21:15,2021-12-30 12:39:29,2021-12-30 12:39:43,ДЭФИР,Группа 2,Артем О,Клиент 503,p031i04ftp01.unix.local -> Нет файлов CDR от П...,Да,4,4
2415,INC00001444482,Инцидент,НОП,3 - Средний,Закрыт,2021-12-29 19:06:49,2021-12-30 14:17:29,2022-01-12 14:18:25,ДЭФИР,Группа 4,Михаил А,Клиент 976,Инцидент из формы обратной связи cloud.rt.ru,Да,4,4
2416,INC00001444867,Инцидент,НОП,3 - Средний,Закрыт,2021-12-30 14:26:08,2021-12-30 14:28:48,2022-01-12 14:30:03,ДЭФИР,Группа 4,Михаил Г,Клиент 104,Заказ 124547,Да,4,4
2418,INC00001445163,Инцидент,НОП,3 - Средний,Закрыт,2021-12-31 07:24:47,2021-12-31 08:19:31,2022-01-01 08:21:29,ДЭФИР,Группа 5,Роман М,Клиент 6,Обращение через AF НОП,Да,4,4


In [ ]:
import pandas as pd
import psycopg2



query = """
    WITH last_date AS (
        SELECT MAX("date") AS dt
        FROM rtk.test_sql_py 
        WHERE "date" < '2022-07-04' -- CURRENT_DATE
    )
    SELECT *
    FROM rtk.test_sql_py tsp 
    JOIN last_date ld ON tsp."date" = ld.dt;
"""


with pd.ExcelWriter(f"result.xlsx", engine='xlsxwriter', datetime_format='dd.mm.yyyy') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    df_dynamic.to_excel(writer, sheet_name='Sheet2', index=False)